In [1]:
!python --version

Python 3.10.9


In [2]:
!nvidia-smi

Thu Jan 26 04:09:11 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:1E.0 Off |                    0 |
| N/A   39C    P8    15W /  70W |      2MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Tue_Mar__8_18:18:20_PST_2022
Cuda compilation tools, release 11.6, V11.6.124
Build cuda_11.6.r11.6/compiler.31057947_0


In [4]:
import torch
torch.__version__

'2.0.0.dev20230124'

In [6]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 79.3 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 770.5/770.5 kB 82.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 kB 44.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 52.4 MB/s eta 0:00:0000:0100:01m


In [7]:
import transformers
transformers.__version__

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'4.26.0'

In [8]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch._dynamo as torchdynamo
import torch

In [9]:
# default cache size needs to be increased to store the many graphs with generative models
torchdynamo.config.cache_size_limit = 512

In [10]:
model_name = "t5-small"

In [11]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model = model.eval().cuda()
tokenizer = AutoTokenizer.from_pretrained(model_name)

/opt/conda/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [12]:
inputs = tokenizer("Generate taxonomy for query: dildo", return_tensors="pt").to('cuda')

# pytorch

## vanilla

In [13]:
%%timeit
outputs = model.generate(**inputs, num_beams=3, num_return_sequences=3)

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


98.8 ms ± 2.54 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
%%timeit
outputs = model.generate(**inputs)

48.8 ms ± 622 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## inference mode

In [15]:
%%timeit
with torch.inference_mode():
    outputs = model.generate(**inputs, num_beams=3, num_return_sequences=3)

87.7 ms ± 854 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [16]:
%%timeit
with torch.inference_mode():
    outputs = model.generate(**inputs)

43.7 ms ± 448 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## fp16 inference mode (slower than without fp16)

In [17]:
%%timeit
with torch.inference_mode(), torch.autocast(dtype=torch.float16, cache_enabled=True, device_type="cuda"):
    outputs = model.generate(**inputs, num_beams=3, num_return_sequences=3)

99.3 ms ± 1.26 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [18]:
%%timeit
with torch.inference_mode(), torch.autocast(dtype=torch.float16, cache_enabled=True, device_type="cuda"):
    outputs = model.generate(**inputs)

51.8 ms ± 796 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## compile

In [27]:
model_compile = torch.compile(model, mode="reduce-overhead")

In [28]:
%%timeit
with torch.inference_mode():
    outputs = model_compile.generate(**inputs, num_beams=3, num_return_sequences=3)

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


89.6 ms ± 841 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [29]:
%%timeit
with torch.inference_mode():
    outputs = model_compile.generate(**inputs)

44.6 ms ± 420 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


# dynamo

In [30]:
model.generate2 = torchdynamo.optimize("inductor")(model.generate)

In [31]:
torch._dynamo.config.verbose=True

In [32]:
inputs

{'input_ids': tensor([[ 6939,  2206,  1104,    32,  3114,    63,    21, 11417,    10,     3,
            26,   173,    26,    32,     1]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [33]:
# dynamo warm up
with torch.inference_mode():
    outputs = model.generate2(inputs=inputs["input_ids"])

InternalTorchDynamoError: 